In [55]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
import json
import os
import time
import random
import pandas as pd

In [46]:
url = 'https://supermarket-scrape-website.vercel.app/'

In [47]:
response = requests.get(f'{url}/api/category')
temp_categories = json.loads(response.content)
temp_categories = temp_categories['categories']

In [48]:
categories = []
for category in temp_categories:
    categories.append(category['key'])
print(categories)

['confectionery', 'delicatessen', 'household', 'winecellar', 'fruitsandvegetables', 'organic', 'pets', 'chilledanddairy', 'fish', 'butcher']


In [49]:
folder_path = './json_dumps/'
for category in categories:
    if not os.path.exists(folder_path+category):
        os.makedirs(folder_path+category)

In [50]:
def getPageContent(category, page):
    res = requests.get(f'{url}/api/product?category={category}&page={page}')
    if res.status_code==200:
        return json.loads(res.content)
    return None


In [51]:
def writeJsonToFile(category, page, file):
    with open (f'./json_dumps/{category}/{page}.json', 'w') as json_file:
        json.dump(file, json_file)

In [52]:
for category in categories:
    page = 1
    pageEnd = False
    while not pageEnd:
        print (f'Scraping {category} page {page}')
        content = getPageContent(category, page)
        writeJsonToFile(category, page, content)
        pageEnd = len(content['products']) == 0
        page+=1
        sleep_duration = random.uniform(1, 5)
        time.sleep(sleep_duration)

Scraping confectionery page 1
Scraping confectionery page 2
Scraping confectionery page 3
Scraping confectionery page 4
Scraping confectionery page 5
Scraping delicatessen page 1
Scraping delicatessen page 2
Scraping delicatessen page 3
Scraping delicatessen page 4
Scraping delicatessen page 5
Scraping household page 1
Scraping household page 2
Scraping household page 3
Scraping household page 4
Scraping household page 5
Scraping winecellar page 1
Scraping winecellar page 2
Scraping winecellar page 3
Scraping winecellar page 4
Scraping winecellar page 5
Scraping fruitsandvegetables page 1
Scraping fruitsandvegetables page 2
Scraping fruitsandvegetables page 3
Scraping fruitsandvegetables page 4
Scraping fruitsandvegetables page 5
Scraping organic page 1
Scraping organic page 2
Scraping organic page 3
Scraping organic page 4
Scraping organic page 5
Scraping pets page 1
Scraping pets page 2
Scraping pets page 3
Scraping pets page 4
Scraping pets page 5
Scraping chilledanddairy page 1
Scr

In [57]:
source_folder = './json_dumps/'
categories = os.listdir(source_folder)

if not os.path.exists('json_combined'):
    os.makedirs('json_combined')


In [62]:
for category in categories:
    print(f'Combining {category}')
    all_product_details = []
    files = os.listdir(f'{source_folder}{category}')
    for file in files:
        with open(f'{source_folder}{category}/{file}', 'r') as file:
            data = json.load(file)
            for item in data['products']:
                all_product_details.append(item)
    df = pd.DataFrame(all_product_details)
    df.to_csv(f'./json_combined/{category}.csv', index=False)

Combining butcher
Combining chilledanddairy
Combining confectionery
Combining delicatessen
Combining fish
Combining fruitsandvegetables
Combining household
Combining organic
Combining pets
Combining winecellar


In [63]:
source_folder = './json_combined/'
files = os.listdir(source_folder)
combined_df = pd.concat([pd.read_csv(os.path.join(source_folder, file)) for file in files])


In [64]:
if not os.path.exists('json_final'):
    os.makedirs('json_final')

In [65]:
combined_df.to_csv('./json_final/rest_products.csv', index=False)